# Minacce Informatiche e Perdite Economiche Globali 

L'analisi si focalizza su dati riguardanti **minacce informatiche globali** e le **perdite economiche stimate** ad esse associate, nel periodo compreso tra il **2015 e il 2024**.  
L’obiettivo è fornire una panoramica dell’impatto economico della criminalità informatica.

### Struttura del dataset `Global_Cybersecurity_Threats_2015-2024`

Il file CSV è organizzato con una riga per ogni osservazione (evento o aggregazione annuale), e presenta le seguenti colonne principali:

- **`country`**: nome del paese a cui è attribuito l’impatto dell’attacco informatico.
- **`year`**: anno dell’evento registrato (dal 2015 al 2024).
- **`threat_type`**: tipologia di minaccia informatica coinvolta (es. *Malware, Ransomware, Phishing, Data Breach*, ecc.).
- **`sector_affected`**: settore economico colpito (es. *Finance, Healthcare, Government*, ecc.).
- **`financial_loss_(in_million_$)`**: stima economica della perdita subita a causa della minaccia, espressa in **milioni di dollari statunitensi (USD)**.
- **`incident_count`** *(se presente)*: numero totale di incidenti segnalati per quella combinazione di anno/paese/tipo di attacco.


In [56]:
import plotly.express as px          
import pandas as pd                 
import plotly.graph_objects as go
df = pd.read_csv("Global_Cybersecurity_Threats_2015-2024.csv")


### Evoluzione degli Attacchi per Settore Economico 

Questo grafico a barre impilate mostra **l’andamento temporale degli attacchi informatici**, suddivisi per **settore economico colpito**.  Ogni barra rappresenta un anno, mentre i segmenti colorati al suo interno indicano il numero di attacchi registrati per ciascun settore (ordinati in base al numero di attacchi).




In [57]:
# raggruppamento dei dati per anno e settore
grouped = df.groupby(['Year', 'Target Industry']).size().reset_index(name='Attacchi')

# pivot (anno x settore)
pivot = grouped.pivot(index='Year', columns='Target Industry', values='Attacchi').fillna(0)

# Colori
color_palette = px.colors.qualitative.Plotly
industries = pivot.columns.tolist()
color_map = {ind: color_palette[i % len(color_palette)] for i, ind in enumerate(industries)}
fig = go.Figure()
legend_shown = set()
for year in pivot.index:
    values = pivot.loc[year]
    sorted_sectors = values.sort_values().index.tolist()
    cumulative_base = 0
    for sector in sorted_sectors:
        val = values[sector]
        if val == 0:
            continue
        fig.add_trace(go.Bar(
            x=[year],
            y=[val],
            base=[cumulative_base],
            name=sector,
            marker_color=color_map[sector],
            showlegend=sector not in legend_shown,
            text=[val],
            textposition='inside',
            textfont=dict(
                family='Segoe UI',
                size=24,  
                color='white'
            )
        ))
        legend_shown.add(sector)
        cumulative_base += val
# stile 
fig.update_layout(
    barmode='stack',
    template='plotly_dark',
    width=1780,   # larghezza figura (in pixel)
    height=550,   # altezza figura (in pixel)
    plot_bgcolor='#050b18',
    paper_bgcolor='#050b18',
    font=dict(
        family='Segoe UI',
        size=28,
        color='white'
    ),
    xaxis=dict(
        title=dict(text='Anno', font=dict(family='Segoe UI', size=32, color='white')),
        tickfont=dict(family='Segoe UI', size=28, color='white')
    ),
    yaxis=dict(
        title=dict(text='Numero di attacchi', font=dict(family='Segoe UI', size=32, color='white')),
        tickfont=dict(family='Segoe UI', size=28, color='white')
    ),
    legend=dict(
        font=dict(family='Segoe UI', size=18, color='white'),
        traceorder='normal'
    ),
)
fig.show()

### Distribuzione degli Attacchi per Settore e Tipologia

Questo grafico mostra **come si distribuiscono i diversi tipi di attacchi informatici all’interno di ciascun settore economico**. Ogni subplot rappresenta un **settore bersaglio**. All’interno di ogni riquadro, le barre indicano **quanti attacchi sono stati registrati** per ciascuna tipologia (es. *Ransomware, Data Breach, Phishing*, ecc.).




In [58]:
import plotly.express as px
# rimozione del settore "Telecommunications"
df = df[df['Target Industry'] != 'Telecommunications']
# raggruppamento per settore e attacco
grouped = (
    df.groupby(['Target Industry', 'Attack Type'])
    .size()
    .reset_index(name='Count')
)
# grafico a barre
fig = px.bar(
    grouped,
    x="Attack Type",
    y="Count",
    color="Attack Type",
    facet_col="Target Industry",
    facet_col_wrap=3,
    facet_col_spacing=0.07,
    facet_row_spacing=0.17,
    title="",
    template="plotly_dark"
)
fig.update_layout(
    paper_bgcolor='#050b18',
    plot_bgcolor='#050b18',
    height=1000,
    width=1800,
    font=dict(family='Segoe UI', color='white', size=22),
    legend=dict(
        font=dict(family='Segoe UI', size=20, color='white')
    )
)

# aggiornamento dei titoli dei facet 
fig.for_each_annotation(lambda a: a.update(
    text=a.text.split("=")[-1],
    font=dict(family='Segoe UI', size=22, color='white')
))
# rimpzione titoli e tick delle assi X e Y
fig.update_xaxes(
    showticklabels=False,
    title=None
)
fig.update_yaxes(
    showticklabels=True,  
    title=None
)
fig.show()

### Tipo di Attacco Dominante per Anno 

Questo grafico mette in evidenza, per ciascun anno tra il 2015 e il 2024, **la tipologia di attacco informatico che ha registrato il maggior numero di incidenti** a livello globale. Ogni barra rappresenta un anno, e mostra:

- **Il numero totale di incidenti** legati al tipo di attacco più frequente in quell'anno.
- Il **nome dell’attacco dominante** visualizzato come etichetta sopra ogni barra.





In [59]:
# estrazione top Attack Type per ogni anno
top_attack_per_year = (
    df.groupby(["Year", "Attack Type"])
      .size()
      .reset_index(name="Incident Count")
      .sort_values(["Year", "Incident Count"], ascending=[True, False])
      .drop_duplicates(subset="Year", keep="first")
      .sort_values("Year")
)
# colori in base ai tipi di attacco
attack_types = top_attack_per_year["Attack Type"].unique().tolist()
custom_colors = {
    attack_types[i]: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
    for i in range(len(attack_types))
}
#grafico a barre

fig = px.bar(
    top_attack_per_year,
    x="Year",
    y="Incident Count",
    color="Attack Type",
    text="Attack Type",
    color_discrete_map=custom_colors  # qui usiamo i colori personalizzati
)
fig.update_traces(
    textposition='outside',
    textfont=dict(size=28, family='Segoe UI', color='white')
)
fig.update_layout(
    width=1780,    
    height=550,   
    paper_bgcolor='#050b18',
    plot_bgcolor='#050b18',
    font=dict(family='Segoe UI', color='white', size=30),

    xaxis=dict(
        title=None,
        showticklabels=True,  # mostra tutti gli anni
        tickmode='array',
        tickvals=top_attack_per_year["Year"],
        ticktext=top_attack_per_year["Year"].astype(str)
    ),
    yaxis=dict(
        title=dict(text='Numero di incidenti', font=dict(family='Segoe UI', size=32, color='white')),
        tickfont=dict(family='Segoe UI', size=28, color='white'),
        showgrid=True, gridcolor='rgba(255,255,255,0.15)', gridwidth=1, zeroline=False
    ),
    legend=dict(title='', font=dict(family='Segoe UI', size=26, color='white')),
    title=None
)
fig.update_traces(
    textposition='outside',
    textfont=dict(size=28, family='Segoe UI', color='white'),
    cliponaxis=False
)
fig.show()
